# Climatology calculations

In [ ]:
# Imports
from earthkit import transforms as ekt
from earthkit import data as ekd
from earthkit.data.testing import earthkit_remote_test_data_file
ekd.settings.set("cache-policy", "user")

## Load some test data

In this example we will use hourly ERA5 2m temperature data on a 0.5x0.5 spatial grid for the year 2015 as
our physical data; and we will use the NUTS geometries which are stored in a geojson file.

All `earthkit-transforms` methods can be called with `earthkit-data` objects (Readers and Wrappers) or with a pre-loaded `xarray`. To reduce the number of conversions in the example, we will convert to xarray in the first cell and use that data object for all subsequent steps.

In [ ]:
# Get some demonstration ERA5 data, this could be any url or path to an ERA5 grib or netCDF file.
remote_era5_file = earthkit_remote_test_data_file("era5_temperature_france_2015_2016_2017_3deg.grib")
era5_data = ekd.from_source("url", remote_era5_file)

# convert to xarray to save repeated conversion in further steps
era5_xr = era5_data.to_xarray(time_dim_mode="valid_time")
era5_xr

## Calculate the climatologies of the ERA5 data

Monthly mean

In [ ]:
climatology_monthly_mean = ekt.climatology.monthly_mean(era5_xr)
# # The following line would also work, but we have already converted the data to xarray,
# #  so do not need to do it again.
# climatology = ek_aggregate.climatology.monthly_mean(era5_xr)
climatology_monthly_mean

Climatology of the daily mean:

In [ ]:
climatology_daily_mean = ekt.climatology.daily_mean(era5_xr)
climatology_daily_mean

Repeat for the monthly maximum, minimum  and standard deviation

In [ ]:
clim_max = ekt.climatology.monthly_max(era5_xr)
clim_min = ekt.climatology.monthly_min(era5_xr)
clim_std = ekt.climatology.monthly_std(era5_xr)
clim_std

### Quantiles

Please note the api for quantiles is slightly different, it requires an additional argument `q` which is a list of the quantiles to return.

Additionally, the returned object has `quantiles` dimension which is for each of the quantiles returned.

In [ ]:
quantiles = ekt.climatology.quantiles(era5_xr, [0.1, 0.5, 0.9], frequency='month')
quantiles

## Plot the output for a random location

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import xarray as xr
isel_kwargs = {"latitude":2, "longitude":4}

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

for data in [climatology_monthly_mean, clim_min, clim_max]:
    var_name = list(data.data_vars.keys())[0]
    c_point = data[var_name].isel(**isel_kwargs)
    c_time = [datetime(2000,i,15) for i in c_point.month.values]
    ax.plot(c_time, c_point.values.flat, label=f'Monthly {var_name}')


data = climatology_daily_mean
var_name = list(data.data_vars.keys())[0]
c_point = data[var_name].isel(**isel_kwargs)
c_time = [datetime(2000,1,1)+timedelta(days=int(i-1)) for i in c_point.dayofyear.values]
ax.plot(c_time, c_point.values.flat, label=f'Daily {var_name}')

ax.legend()
